In [15]:
import numpy as np
from itertools import chain
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import torch
from sklearn.metrics import confusion_matrix, mean_squared_error
import os
import pickle
import time
from Graphs import matrix_to_graph, graph_to_matrix, ErdosRenyiGraph,dRegularGraph
from ShortestDistanceAlgorithms import shortestDistance_allNodes_networkx, shortestDistance_allNodes_Bourgain, shortestDistance_allNodes_Sarma, onlineShortestPath_Bourgain, onlineShortestPath_Sarma
from Models import build

In [16]:
def predict(gpu_bool,model,criterion_type,samples_x,samples_edge_index=[],samples_weights=[]):
    
    if len(samples_edge_index) > 0:
        if len(samples_edge_index) == 1:
            flag = True
        else:
            flag = False
    
    y_pred = []
    if gpu_bool:
        model = model.to('cuda:1')
    model.eval()
    with torch.no_grad():
        if model.out_channels == 1:
            if model.name == 'mlp':
                for x in samples_x:
                    if gpu_bool:
                        x = x.to('cuda:1')
                    pred_all = []
                    for j in range(x.shape[1]):
                        out = model(x[:,j].reshape(len(x[:,j]),1))  # Perform a single forward pass.
                        if criterion_type in ['bce','ce','multimargin']:
                            pred = out.argmax(dim=1) #  Use the class with highest probability.
                        elif criterion_type in ['mse','l2','l1']:
                            pred = out.squeeze()
                        else:
                            pred = torch.round(out.squeeze())
                        pred_all.append(pred.cpu())
                    y_pred.append(np.array(pred_all).T)
            elif len(samples_weights) == 0:
                if flag:
                    edge_index = samples_edge_index[-1]
                    if gpu_bool:
                        edge_index = edge_index.to('cuda:1')
                    for x in samples_x:
                        if gpu_bool:
                            x = x.to('cuda:1')
                        pred_all = []
                        for j in range(x.shape[1]):
                            out = model(x[:,j].reshape(len(x[:,j]),1),edge_index)  # Perform a single forward pass.
                            if criterion_type in ['bce','ce','multimargin']:
                                pred = out.argmax(dim=1) #  Use the class with highest probability.
                            elif criterion_type in ['mse','l2','l1']:
                                pred = out.squeeze()
                            else:
                                pred = torch.round(out.squeeze())
                            pred_all.append(pred.cpu())
                        y_pred.append(np.array(pred_all).T)
                else:
                    for x,edge_index in list(zip(samples_x,samples_edge_index)):
                        if gpu_bool:
                            x = x.to('cuda:1')
                            edge_index = edge_index.to('cuda:1')
                        pred_all = []
                        for j in range(x.shape[1]):
                            out = model(x[:,j].reshape(len(x[:,j]),1),edge_index)  # Perform a single forward pass.
                            if criterion_type in ['bce','ce','multimargin']:
                                pred = out.argmax(dim=1) #  Use the class with highest probability.
                            elif criterion_type in ['mse','l2','l1']:
                                pred = out.squeeze()
                            else:
                                pred = torch.round(out.squeeze())
                            pred_all.append(pred.cpu())
                        y_pred.append(np.array(pred_all).T)
            else:
                if flag:
                    edge_index = samples_edge_index[-1]
                    weights = samples_weights[-1]
                    if gpu_bool:
                        edge_index = edge_index.to('cuda:1')
                        weights = weights.to('cuda:1')
                    for x in samples_x:
                        if gpu_bool:
                            x = x.to('cuda:1')
                        pred_all = []
                        for j in range(x.shape[1]):
                            out = model(x[:,j].reshape(len(x[:,j]),1),edge_index,weights)  # Perform a single forward pass.
                            if criterion_type in ['bce','ce','multimargin']:
                                pred = out.argmax(dim=1) #  Use the class with highest probability.
                            elif criterion_type in ['mse','l2','l1']:
                                pred = out.squeeze()
                            else:
                                pred = torch.round(out.squeeze())
                            pred_all.append(pred.cpu())
                        y_pred.append(np.array(pred_all).T)
                else:
                    for x,edge_index,weights in list(zip(samples_x,samples_edge_index,samples_weights)):
                        if gpu_bool:
                            x = x.to('cuda:1')
                            edge_index = edge_index.to('cuda:1')
                            weights = weights.to('cuda:1')
                        pred_all = []
                        for j in range(x.shape[1]):
                            out = model(x[:,j].reshape(len(x[:,j]),1),edge_index,weights)  # Perform a single forward pass.
                            if criterion_type in ['bce','ce','multimargin']:
                                pred = out.argmax(dim=1) #  Use the class with highest probability.
                            elif criterion_type in ['mse','l2','l1']:
                                pred = out.squeeze()
                            else:
                                pred = torch.round(out.squeeze())
                            pred_all.append(pred.cpu())
                        y_pred.append(np.array(pred_all).T)
        else:
            if model.name == 'mlp':
                for x in samples_x:
                    if gpu_bool:
                        x = x.to('cuda:1')
                    out = model(x)  # Perform a single forward pass.
                    if criterion_type in ['bce','ce','multimargin']:
                        pred = out.argmax(dim=1) #  Use the class with highest probability.
                    elif criterion_type in ['mse','l2','l1']:
                        pred = out.squeeze()
                    else:
                        pred = torch.round(out.squeeze())
                    y_pred.append(pred.cpu())
            elif len(samples_weights) == 0:
                if flag:
                    edge_index = samples_edge_index[-1]
                    if gpu_bool:
                        edge_index = edge_index.to('cuda:1')
                    for x in samples_x:
                        if gpu_bool:
                            x = x.to('cuda:1')
                        out = model(x,edge_index)  # Perform a single forward pass.
                        if criterion_type in ['bce','ce','multimargin']:
                            pred = out.argmax(dim=1) #  Use the class with highest probability.
                        elif criterion_type in ['mse','l2','l1']:
                            pred = out.squeeze()
                        else:
                            pred = torch.round(out.squeeze())
                        y_pred.append(pred.cpu())
                else:
                    for x,edge_index in list(zip(samples_x,samples_edge_index)):
                        if gpu_bool:
                            x = x.to('cuda:1')
                            edge_index = edge_index.to('cuda:1')
                        out = model(x,edge_index)  # Perform a single forward pass.
                        if criterion_type in ['bce','ce','multimargin']:
                            pred = out.argmax(dim=1) #  Use the class with highest probability.
                        elif criterion_type in ['mse','l2','l1']:
                            pred = out.squeeze()
                        else:
                            pred = torch.round(out.squeeze())
                        y_pred.append(pred.cpu())
            else:
                if flag:
                    edge_index = samples_edge_index[-1]
                    weights = samples_weights[-1]
                    if gpu_bool:
                        edge_index = edge_index.to('cuda:1')
                        weights = weights.to('cuda:1')
                    for x in samples_x:
                        if gpu_bool:
                            x = x.to('cuda:1')
                        out = model(x,edge_index,weights)  # Perform a single forward pass.
                        if criterion_type in ['bce','ce','multimargin']:
                            pred = out.argmax(dim=1) #  Use the class with highest probability.
                        elif criterion_type in ['mse','l2','l1']:
                            pred = out.squeeze()
                        else:
                            pred = torch.round(out.squeeze())
                        y_pred.append(pred.cpu())
                else:
                    for x,edge_index,weights in list(zip(samples_x,samples_edge_index,samples_weights)):
                        if gpu_bool:
                            x = x.to('cuda:1')
                            edge_index = edge_index.to('cuda:1')
                            weights = weights.to('cuda:1')
                        out = model(x,edge_index,weights)  # Perform a single forward pass.
                        if criterion_type in ['bce','ce','multimargin']:
                            pred = out.argmax(dim=1) #  Use the class with highest probability.
                        elif criterion_type in ['mse','l2','l1']:
                            pred = out.squeeze()
                        else:
                            pred = torch.round(out.squeeze())
                        y_pred.append(pred.cpu())
    model = model.to('cpu')
    return y_pred

def predict_allBatches(model,criterion_type,samples):
    gpu_bool = torch.cuda.is_available()
    y_pred_train = predict(gpu_bool, model, criterion_type, samples[0][0], samples[0][2], samples[0][3])
    y_pred_val = predict(gpu_bool, model, criterion_type, samples[1][0], samples[1][2], samples[1][3])
    y_pred_test = predict(gpu_bool, model, criterion_type, samples[2][0], samples[2][2], samples[2][3])
    return y_pred_train,y_pred_val,y_pred_test

In [17]:
def offlineSample_GNN(model,criterion_type,G,u,samples_edge_index,samples_weights):

    n_nodes = len(G.nodes())
    if n_nodes <= 1:
        return None, set()
    r = int(np.floor(np.log(n_nodes)))
    sample_sets = [np.random.choice(G.nodes(),size=2**i,replace=False) for i in range(r+1)]

    gpu_bool = torch.cuda.is_available()
    out = model.out_channels
    x = np.zeros((n_nodes,out))
    x[u,0] = 1
    if out > 1:
        extra_seeds = np.random.choice(G.nodes(),size=out-1,replace=True)
        for i in range(out-1):
            x[extra_seeds[i],i+1] = 1
    samples_x = [torch.tensor(x.astype(np.float32), requires_grad=True)]
    y_pred = predict(gpu_bool, model, criterion_type, samples_x, samples_edge_index, samples_weights)[0][:,0]
    distances = [[y_pred[i] for i in S] for S in sample_sets]
    index = [d.index(min(d)) for d in distances]
    closest_points = set([(sample_sets[i][index[i]],distances[i][index[i]]) for i in range(r+1)])
    return closest_points,set(np.concatenate(sample_sets))

def offlineSketch_GNN(model,criterion_type,G,u,k,samples_edge_index,samples_weights):
    closest_points,sample_sets = offlineSample_GNN(model,criterion_type,G,u,samples_edge_index,samples_weights)
    for i in range(k):
        closest_points_new,sample_sets_new = offlineSample_GNN(model,criterion_type,G,u,samples_edge_index,samples_weights)
        closest_points = closest_points.union(closest_points_new)
        sample_sets = sample_sets.union(sample_sets_new)
    #print(np.array(list(closest_points)).shape[0])
    return np.array(list(closest_points)),np.array(list(sample_sets))

def onlineShortestPath_Sarma_GNN(model1,model2,criterion_type,G,u,v,k,samples_edge_index,samples_weights): ## upper bound
    ## if undirected, model1 = model2
    ## else, model1 calculates distances from u to each k and model2 calculates distances from eack k to v
    sketch_u,_ = offlineSketch_GNN(model1,criterion_type,G,u,k,samples_edge_index,samples_weights)
    sketch_v,_ = offlineSketch_GNN(model2,criterion_type,G,v,k,samples_edge_index,samples_weights)
    if sketch_u.shape[0] != 0 and sketch_v.shape[0] != 0:
        common_nodes = [w for w in sketch_u[:,0] if w in sketch_v[:,0]]
        while None in common_nodes:
            common_nodes.remove(None)
        min_dist = float('inf')
        for w in common_nodes:
            dist = sketch_u[sketch_u[:, 0] == w][0,1] + sketch_v[sketch_v[:, 0] == w][0,1]
            if dist < min_dist:
                min_dist = dist
        return min_dist
    else:
        return float('inf')

def onlineShortestPath_Bourgain_GNN(model1,model2,criterion_type,G,u,v,samples_edge_index,samples_weights): ## lower bound

    n_nodes = len(G.nodes())
    if n_nodes <= 1:
        return None, set()
    r = int(np.floor(np.log(n_nodes)))
    sample_sets = [np.random.choice(G.nodes(),size=2**i,replace=False) for i in range(r+1)]

    gpu_bool = torch.cuda.is_available()
    if model1 == model2: # set model1 = model2 if undirected, model used need to be trained on undirected graphs
        out = model1.out_channels
        x = np.zeros((n_nodes,out))
        x[u,0] = 1
        if out > 1:
            x[v,1] = 1
            extra_seeds = np.random.choice(G.nodes(),size=out-2,replace=True)
            for i in range(out-2):
                x[extra_seeds[i],i+2] = 1
            samples_x = [torch.tensor(x.astype(np.float32), requires_grad=True)]
            y_pred = predict(gpu_bool, model1, criterion_type, samples_x, samples_edge_index, samples_weights)[0]
            y_pred_u = y_pred[:,0]
            y_pred_v = y_pred[:,1]
        if out == 1:
            samples_x = [torch.tensor(x.astype(np.float32), requires_grad=True)]
            y_pred_u = predict(gpu_bool, model1, criterion_type, samples_x, samples_edge_index, samples_weights)[0][:,0]
            x = np.zeros((n_nodes,out))
            x[v,0] = 1
            samples_x = [torch.tensor(x.astype(np.float32), requires_grad=True)]
            y_pred_v = predict(gpu_bool, model1, criterion_type, samples_x, samples_edge_index, samples_weights)[0][:,0]
        d_u_S = np.array([min([y_pred_u[i] for i in S]) for S in sample_sets])
        d_v_S = np.array([min([y_pred_v[i] for i in S]) for S in sample_sets])
        return np.max(np.abs(d_u_S-d_v_S))
    else:
        out = model1.out_channels
        x = np.zeros((n_nodes,out))
        x[u,0] = 1
        if out > 1:
            extra_seeds = np.random.choice(G.nodes(),size=out-1,replace=True)
            for i in range(out-1):
                x[extra_seeds[i],i+1] = 1
        samples_x = [torch.tensor(x.astype(np.float32), requires_grad=True)]
        y_pred_u_k = predict(gpu_bool, model1, criterion_type, samples_x, samples_edge_index, samples_weights)[0][:,0]
        x = np.zeros((n_nodes,out))
        x[v,0] = 1
        if out > 1:
            extra_seeds = np.random.choice(G.nodes(),size=out-1,replace=True)
            for i in range(out-1):
                x[extra_seeds[i],i+1] = 1
        samples_x = [torch.tensor(x.astype(np.float32), requires_grad=True)]
        y_pred_v_k = predict(gpu_bool, model1, criterion_type, samples_x, samples_edge_index, samples_weights)[0][:,0]
        out = model2.out_channels
        x = np.zeros((n_nodes,out))
        x[u,0] = 1
        if out > 1:
            extra_seeds = np.random.choice(G.nodes(),size=out-1,replace=True)
            for i in range(out-1):
                x[extra_seeds[i],i+1] = 1
        samples_x = [torch.tensor(x.astype(np.float32), requires_grad=True)]
        y_pred_k_u = predict(gpu_bool, model2, criterion_type, samples_x, samples_edge_index, samples_weights)[0][:,0]
        x = np.zeros((n_nodes,out))
        x[v,0] = 1
        if out > 1:
            extra_seeds = np.random.choice(G.nodes(),size=out-1,replace=True)
            for i in range(out-1):
                x[extra_seeds[i],i+1] = 1
        samples_x = [torch.tensor(x.astype(np.float32), requires_grad=True)]
        y_pred_k_v = predict(gpu_bool, model2, criterion_type, samples_x, samples_edge_index, samples_weights)[0][:,0]
        d_u_S = np.array([min([y_pred_u_k[i] for i in S]) for S in sample_sets])
        d_v_S = np.array([min([y_pred_v_k[i] for i in S]) for S in sample_sets])
        d_S_u = np.array([min([y_pred_k_u[i] for i in S]) for S in sample_sets])
        d_S_v = np.array([min([y_pred_k_v[i] for i in S]) for S in sample_sets])
        return max([0,np.max(d_S_v-d_S_u),np.max(d_u_S-d_v_S)])

def shortestDistance_allNodes_Sarma_GNN(model1,model2,criterion_type,G,u,k,samples_edge_index,samples_weights):
    ## if undirected, model1 = model2
    ## else, model1 calculates distances from u to each k and model2 calculates distances from eack k to v
    distances = np.zeros(len(G.nodes()))
    for v in range(len(G.nodes())):
        if u != v:
            distances[v] = onlineShortestPath_Sarma_GNN(model1,model2,criterion_type,G,u,v,k,samples_edge_index,samples_weights)
    return distances

def shortestDistance_allNodes_Bourgain_GNN(model1,model2,criterion_type,G,u,samples_edge_index,samples_weights):
    ## if undirected, model1 = model2
    ## else, model1 calculates distances from u to each k and model2 calculates distances from eack k to v
    distances = np.zeros(len(G.nodes()))
    for v in range(len(G.nodes())):
        if u != v:
            distances[v] = onlineShortestPath_Bourgain_GNN(model1,model2,criterion_type,G,u,v,samples_edge_index,samples_weights)
    return distances

In [1]:
def generateGraphs(num_graphs,function,*args,**kwargs):
    graphs = []
    is_directed =[]
    is_weighted = []
    graph_sizes = []
    k = 0
    n_rejected1 = 0
    n_rejected2 = 0
    while k < num_graphs:
        try:
            object,directed,weighted = function(*args,**kwargs)
            n = len(object.nodes())
            components = list(nx.strongly_connected_components(object))
            largest_component = max(components, key=len)
            n_nodes = len(largest_component)
            r = int(np.floor(np.sqrt(n)))
            if n_nodes >= max(r,10):
                object = object.subgraph(largest_component)
                object = nx.relabel_nodes(object, {node: index for index, node in enumerate(object.nodes())})
                graphs.append(object)
                is_directed.append(directed)
                is_weighted.append(weighted)
                graph_sizes.append(n_nodes)
                k += 1
            else:
                n_rejected2 += 1
        except:
            n_rejected1 += 1
    print('Number of graphs rejected because Bourgain\'s and Sarma\'s algorithms yield errors: ',n_rejected1)
    print('Number of graphs rejected because the largest component has insufficient size: ',n_rejected2)
    return graphs,is_directed,is_weighted,min(graph_sizes)

def estimate_AllShortestDistances(graph_data,model1,model2,criterion_type,calculate_Bourgain_Sarma = True,print_summary=False):
    
    graphs = graph_data[0]
    is_directed = graph_data[1]
    is_weighted = graph_data[2]

    dur = np.zeros((9,len(graphs)))
    mse = np.zeros((9,len(graphs)))

    for i in range(len(graphs)):

        #print(i)

        G = graphs[i]
        directed = is_directed[i]
        weighted = is_weighted[i]

        samples_edge_index = [torch.tensor(np.array(list(G.edges())).T).to(torch.int64)]
        if weighted:
            samples_weights = [torch.tensor(list(nx.get_edge_attributes(G,'weight').values())).to(torch.float32)]
        else:
            samples_weights = []

        matrix = graph_to_matrix(G)
        n_nodes = matrix.shape[0]
        M = n_nodes

        if directed:
    
            start_time = time.time()
            y_actual = np.zeros((n_nodes,n_nodes))
            for u in range(n_nodes):
                y_actual[:,u] = shortestDistance_allNodes_networkx(G,u,weight='weight')
            end_time = time.time()
            dur[0,i] = end_time - start_time

            if calculate_Bourgain_Sarma:

                start_time = time.time()
                y_Bourgain = np.zeros((n_nodes,n_nodes))
                for u in range(n_nodes):
                    y_Bourgain[:,u] = shortestDistance_allNodes_Bourgain(matrix,u,directed)
                y_Bourgain = np.where(y_Bourgain == float('inf'), M, y_Bourgain)
                end_time = time.time()
                dur[1,i] = end_time - start_time
                mse[1,i] = mean_squared_error(y_actual, y_Bourgain)

                start_time = time.time()
                y_Sarma1 = np.zeros((n_nodes,n_nodes))
                for u in range(n_nodes):
                    y_Sarma1[:,u] = shortestDistance_allNodes_Sarma(matrix,u,1,directed)
                y_Sarma1 = np.where(y_Sarma1 == float('inf'), M, y_Sarma1)
                end_time = time.time()
                dur[2,i] = end_time - start_time
                mse[2,i] = mean_squared_error(y_actual, y_Sarma1)

                start_time = time.time()
                y_Sarma2 = np.zeros((n_nodes,n_nodes))
                for u in range(n_nodes):
                    y_Sarma2[:,u] = shortestDistance_allNodes_Sarma(matrix,u,2,directed)
                y_Sarma2 = np.where(y_Sarma2 == float('inf'), M, y_Sarma2)
                end_time = time.time()
                dur[3,i] = end_time - start_time
                mse[3,i] = mean_squared_error(y_actual, y_Sarma2)

                start_time = time.time()
                y_Sarma3 = np.zeros((n_nodes,n_nodes))
                for u in range(n_nodes):
                    y_Sarma3[:,u] = shortestDistance_allNodes_Sarma(matrix,u,3,directed)
                y_Sarma3 = np.where(y_Sarma3 == float('inf'), M, y_Sarma3)
                end_time = time.time()
                dur[4,i] = end_time - start_time
                mse[4,i] = mean_squared_error(y_actual, y_Sarma3)

            start_time = time.time()
            y_Bourgain_GNN = np.zeros((n_nodes,n_nodes))
            for u in range(n_nodes):
                #print(u)
                y_Bourgain_GNN[:,u] = shortestDistance_allNodes_Bourgain_GNN(model1,model2,criterion_type,G,u,samples_edge_index,samples_weights)
            y_Bourgain_GNN = np.where(y_Bourgain_GNN == float('inf'), M, y_Bourgain_GNN)
            end_time = time.time()
            dur[5,i] = end_time - start_time
            mse[5,i] = mean_squared_error(y_actual, y_Bourgain_GNN)

            start_time = time.time()
            y_Sarma1_GNN = np.zeros((n_nodes,n_nodes))
            for u in range(n_nodes):
                y_Sarma1_GNN[:,u] = shortestDistance_allNodes_Sarma_GNN(model1,model2,criterion_type,G,u,1,samples_edge_index,samples_weights)
            y_Sarma1_GNN = np.where(y_Sarma1_GNN == float('inf'), M, y_Sarma1_GNN)
            end_time = time.time()
            dur[6,i] = end_time - start_time
            mse[6,i] = mean_squared_error(y_actual, y_Sarma1_GNN)

            start_time = time.time()
            y_Sarma2_GNN = np.zeros((n_nodes,n_nodes))
            for u in range(n_nodes):
                y_Sarma2_GNN[:,u] = shortestDistance_allNodes_Sarma_GNN(model1,model2,criterion_type,G,u,2,samples_edge_index,samples_weights)
            y_Sarma2_GNN = np.where(y_Sarma2_GNN == float('inf'), M, y_Sarma2_GNN)
            end_time = time.time()
            dur[7,i] = end_time - start_time
            mse[7,i] = mean_squared_error(y_actual, y_Sarma2_GNN)
            
            start_time = time.time()
            y_Sarma3_GNN = np.zeros((n_nodes,n_nodes))
            for u in range(n_nodes):
                y_Sarma3_GNN[:,u] = shortestDistance_allNodes_Sarma_GNN(model1,model2,criterion_type,G,u,3,samples_edge_index,samples_weights)
            y_Sarma3_GNN = np.where(y_Sarma3_GNN == float('inf'), M, y_Sarma3_GNN)
            end_time = time.time()
            dur[8,i] = end_time - start_time
            mse[8,i] = mean_squared_error(y_actual, y_Sarma3_GNN)
        
        else:

            start_time = time.time()
            y_actual = np.zeros((n_nodes,n_nodes))
            for u in range(n_nodes):
                for v in range(i+1,n_nodes):
                    y_actual[v,u] = nx.shortest_path_length(G, u, v, weight='weight')
                    y_actual[u,v] = y_actual[v,i]
            end_time = time.time()
            dur[0,i] = end_time - start_time

            if calculate_Bourgain_Sarma:

                start_time = time.time()
                y_Bourgain = np.zeros((n_nodes,n_nodes))
                for u in range(n_nodes):
                    for v in range(i+1,n_nodes):
                        y_Bourgain[v,u] = onlineShortestPath_Bourgain(matrix,u,v,directed)
                        y_Bourgain[u,v] = y_Bourgain[v,u]
                y_Bourgain = np.where(y_Bourgain == float('inf'), M, y_Bourgain)
                end_time = time.time()
                dur[1,i] = end_time - start_time
                mse[1,i] = mean_squared_error(y_actual, y_Bourgain)

                start_time = time.time()
                y_Sarma1 = np.zeros((n_nodes,n_nodes))
                for u in range(n_nodes):
                    for v in range(i+1,n_nodes):
                        y_Sarma1[v,u] = onlineShortestPath_Sarma(matrix,u,v,1,directed)
                        y_Sarma1[u,v] = y_Sarma1[v,u]
                y_Sarma1 = np.where(y_Sarma1 == float('inf'), M, y_Sarma1)
                end_time = time.time()
                dur[2,i] = end_time - start_time
                mse[2,i] = mean_squared_error(y_actual, y_Sarma1)

                start_time = time.time()
                y_Sarma2 = np.zeros((n_nodes,n_nodes))
                for u in range(n_nodes):
                    for v in range(i+1,n_nodes):
                        y_Sarma2[v,u] = onlineShortestPath_Sarma(matrix,u,v,2,directed)
                        y_Sarma2[u,v] = y_Sarma2[v,u]
                y_Sarma2 = np.where(y_Sarma2 == float('inf'), M, y_Sarma2)
                end_time = time.time()
                dur[3,i] = end_time - start_time
                mse[3,i] = mean_squared_error(y_actual, y_Sarma2)

                start_time = time.time()
                y_Sarma3 = np.zeros((n_nodes,n_nodes))
                for u in range(n_nodes):
                    for v in range(i+1,n_nodes):
                        y_Sarma3[v,u] = onlineShortestPath_Sarma(matrix,u,v,3,directed)
                        y_Sarma3[u,v] = y_Sarma3[v,u]
                y_Sarma3 = np.where(y_Sarma3 == float('inf'), M, y_Sarma3)
                end_time = time.time()
                dur[4,i] = end_time - start_time
                mse[4,i] = mean_squared_error(y_actual, y_Sarma3)

            start_time = time.time()
            y_Bourgain_GNN = np.zeros((n_nodes,n_nodes))
            for u in range(n_nodes):
                for v in range(i+1,n_nodes):
                    y_Bourgain_GNN[v,u] = onlineShortestPath_Bourgain_GNN(model1,model2,criterion_type,G,u,v,samples_edge_index,samples_weights)
                    y_Bourgain_GNN[u,v] = y_Bourgain_GNN[v,u]
            y_Bourgain_GNN = np.where(y_Bourgain_GNN == float('inf'), M, y_Bourgain_GNN)
            end_time = time.time()
            dur[5,i] = end_time - start_time
            mse[5,i] = mean_squared_error(y_actual, y_Bourgain_GNN)

            start_time = time.time()
            y_Sarma1_GNN = np.zeros((n_nodes,n_nodes))
            for u in range(n_nodes):
                for v in range(i+1,n_nodes):
                    y_Sarma1_GNN[v,u] = onlineShortestPath_Sarma_GNN(model1,model2,criterion_type,G,u,v,1,samples_edge_index,samples_weights)
                    y_Sarma1_GNN[u,v] = y_Sarma1_GNN[v,u]
            y_Sarma1_GNN = np.where(y_Sarma1_GNN == float('inf'), M, y_Sarma1_GNN)
            end_time = time.time()
            dur[6,i] = end_time - start_time
            mse[6,i] = mean_squared_error(y_actual, y_Sarma1_GNN)

            start_time = time.time()
            y_Sarma2_GNN = np.zeros((n_nodes,n_nodes))
            for u in range(n_nodes):
                for v in range(i+1,n_nodes):
                    y_Sarma2_GNN[v,u] = onlineShortestPath_Sarma_GNN(model1,model2,criterion_type,G,u,v,2,samples_edge_index,samples_weights)
                    y_Sarma2_GNN[u,v] = y_Sarma2_GNN[v,u]
            y_Sarma2_GNN = np.where(y_Sarma2_GNN == float('inf'), M, y_Sarma2_GNN)
            end_time = time.time()
            dur[7,i] = end_time - start_time
            mse[7,i] = mean_squared_error(y_actual, y_Sarma2_GNN)
            
            start_time = time.time()
            y_Sarma3_GNN = np.zeros((n_nodes,n_nodes))
            for u in range(n_nodes):
                for v in range(i+1,n_nodes):
                    y_Sarma3_GNN[v,u] = onlineShortestPath_Sarma_GNN(model1,model2,criterion_type,G,u,v,3,samples_edge_index,samples_weights)
                    y_Sarma3_GNN[u,v] = y_Sarma3_GNN[v,u]
            y_Sarma3_GNN = np.where(y_Sarma3_GNN == float('inf'), M, y_Sarma3_GNN)
            end_time = time.time()
            dur[8,i] = end_time - start_time
            mse[8,i] = mean_squared_error(y_actual, y_Sarma3_GNN)
        
    mse_mean = np.mean(mse, axis=1)
    dur_mean = np.mean(dur, axis=1)
    
    if print_summary:
        print('networkx: MSE = '+str(mse_mean[0])+', runtime = '+str(dur_mean[0])+' seconds')
        print('Bourgain: MSE = '+str(mse_mean[1])+', runtime = '+str(dur_mean[1])+' seconds')
        print('Sarma, k = 1: MSE = '+str(mse_mean[2])+', runtime = '+str(dur_mean[2])+' seconds')
        print('Sarma, k = 2: MSE = '+str(mse_mean[3])+', runtime = '+str(dur_mean[3])+' seconds')
        print('Sarma, k = 3: MSE = '+str(mse_mean[4])+', runtime = '+str(dur_mean[4])+' seconds')
        print('Bourgain-GNN: MSE = '+str(mse_mean[5])+', runtime = '+str(dur_mean[5])+' seconds')
        print('Sarma-GNN, k = 1: MSE = '+str(mse_mean[6])+', runtime = '+str(dur_mean[6])+' seconds')
        print('Sarma-GNN, k = 2: MSE = '+str(mse_mean[7])+', runtime = '+str(dur_mean[7])+' seconds')
        print('Sarma-GNN, k = 3: MSE = '+str(mse_mean[8])+', runtime = '+str(dur_mean[8])+' seconds')

    return mse_mean, dur_mean

def evaluate_all_algs(model_dir,num_graphs,function,*args,**kwargs):
    all_out_channels = [int(item[9:-4]) for item in os.listdir(model_dir) if item.startswith('model_out')]
    r = int(np.floor(np.sqrt(n)))
    graph_data = generateGraphs(num_graphs,function,*args,**kwargs)
    print('Min graph size: '+str(graph_data[3]))
    if r not in all_out_channels:
        out_selected = [out for out in all_out_channels if out <= graph_data[3]]
        if len(out_selected) > 0:
            r = max(out_selected)
        else:
            raise ValueError(
                'Model not available.'
            )
    model_outr,_,_,_ = build(r, r, 'gcn','mse','adam','cyclic-cosine')
    model_outr.load_state_dict(torch.load(model_dir+'/model_out'+str(r)+'.pth'))
    print('GNN with out_channels = '+str(r))
    mse1,dur1 = estimate_AllShortestDistances(graph_data,model_outr,model_outr,'mse',True)
    if r != 1:
        model_out1,_,_,_ = build(1, 1, 'gcn','mse','adam','cyclic-cosine')
        model_out1.load_state_dict(torch.load(model_dir+'/model_out1.pth'))
        print('GNN with out_channels = 1')
        mse2,dur2 = estimate_AllShortestDistances(graph_data,model_out1,model_out1,'mse',True)
    return [mse1,mse2],[dur1,dur2]

In [ ]:
model_dir = 'outputs2'
num_graphs = 100
graph_sizes = list(np.array(range(1,10))*10)+list(2**np.array(range(9))*100)
lbds = [2,4,6]

if os.path.exists(model_dir+'/results.pkl'):
    with open(model_dir+'/results.pkl', 'rb') as file:
        results = pickle.load(file)
else:
    results = []

for n in graph_sizes:
    for lbd in lbds:
        mse,dur = evaluate_all_algs(model_dir,num_graphs,dRegularGraph,n,lbd)
        results.append([[n,lbd,num_graphs],mse,dur])
        with open(model_dir+'/results.pkl', 'wb') as file:
            pickle.dump(results, file)

In [ ]:
num_graphs = 5
n = 1000
lbd = 1
model_dir = 'outputs1'

def evaluate_all_algs(model_dir,num_graphs,function,*args,**kwargs):
    all_out_channels = [int(item[9:-4]) for item in os.listdir(model_dir) if item.startswith('model_out')]
    r = int(np.floor(np.sqrt(n)))
    graph_data = generateGraphs(num_graphs,function,*args,**kwargs)
    print('Min graph size: '+str(graph_data[3]))
    if r not in all_out_channels:
        r = max([out for out in all_out_channels if out <= graph_data[3]])
    model_outr,_,_,_ = build(r, r, 'gcn','mse','adam','cyclic-cosine')
    model_outr.load_state_dict(torch.load('outputs1/model_out'+str(r)+'.pth'))
    print('GNN with out_channels = '+str(r))
    mse1,dur1 = estimate_AllShortestDistances(graph_data,model_outr,model_outr,'mse',True,True)
    if r != 1:
        model_out1,_,_,_ = build(1, 1, 'gcn','mse','adam','cyclic-cosine')
        model_out1.load_state_dict(torch.load('outputs1/model_out1.pth'))
        print('GNN with out_channels = 1')
        mse2,dur2 = estimate_AllShortestDistances(graph_data,model_out1,model_out1,'mse',True,True)
    return [mse1,mse2],[dur1,dur2]

Number of graphs rejected because Bourgain's and Sarma's algorithms yield errors:  0
Number of graphs rejected because the largest component has insufficient size:  0
0
1
2
3
4
(array([  0.28226895,  20.64235311,  42.62032714,  63.75388665,
        84.7655674 ,  17.52850881,  68.36310229, 102.47579651,
       136.47396355]), array([   0.        ,   20.87908171, 2186.5779133 ,  884.75924051,
        304.30625398,   80.13021451, 2110.94547699,  678.47364176,
        169.09748665]))
0


KeyboardInterrupt: 